In [86]:
import openml
import pandas as pd
import numpy as np

In [87]:
openml_list = openml.datasets.list_datasets()  # returns a dict
datalist = pd.DataFrame.from_dict(openml_list, orient="index")
datalist["name"] = datalist.name.apply(lambda x: x.lower())

/tmp/ipykernel_286230/2787330776.py:1: FutureWarning: Support for `output_format` of 'dict' will be removed in 0.15 and pandas dataframes will be returned instead. To ensure your code will continue to work, use `output_format`='dataframe'.
  openml_list = openml.datasets.list_datasets()  # returns a dict


In [88]:
names = [
    "australian",
    "blood-transfusion-service-center",
    "car",
    "cnae-9",
    "credit-g",
    "jasmine",
    "kc1",
    "kr-vs-kp",
    "vehicle",
    "segment",
    "sylvine",
    "mfeat-factors"
]

In [89]:
info = datalist[datalist.name.apply(lambda x: x in names)]#.fillna(-1)
info["name"] = info["name"].apply(lambda x: x.lower())
replace_id = info[info["name"] == "blood-transfusion-service-center"].index
info.loc[replace_id, "name"] = "blood-transfusion"

def select_latest(df):
    latest = df.version.max()
    return df[df.version == latest]

info = info.groupby("name").apply(select_latest)

info = info.set_index("name")
assert len(info) == 12


/tmp/ipykernel_286230/1466267504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info["name"] = info["name"].apply(lambda x: x.lower())
/home/cherniak/miniconda3/envs/gamlet/lib/python3.10/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [90]:
def calc_majority_class_size(df, name):
    return df["y"].value_counts().max()

def calc_minority_class_size(df, name):
    return df["y"].value_counts().min()

def calc_number_of_classes(df, name):
    return len(df["y"].value_counts())
    # return info.loc[name]["NumberOfClasses"]

def calc_number_of_features(df, name):
    return info.loc["australian"]["NumberOfFeatures"]

def calc_number_of_instances(df, name):
    return len(df)

def calc_number_of_instances_with_missing_values(df, name):
    return df.isna().sum(axis=1).astype(bool).astype(int).sum()

def calc_number_of_missing_values(df, name):
    return df.isna().to_numpy().sum()

def calc_number_of_numeric_features(df, name):
    return info.loc[name]["NumberOfNumericFeatures"]

def calc_number_of_symbolic_features(df, name):
    return info.loc[name]["NumberOfSymbolicFeatures"]

def extract(df, name):
    return {
        "MajorityClassSize": calc_majority_class_size(df, name),
        "MinorityClassSize": calc_minority_class_size(df, name),
        "NumberOfClasses": calc_number_of_classes(df, name),
        "NumberOfFeatures": calc_number_of_features(df, name),
        "NumberOfInstances": calc_number_of_instances(df, name),
        "NumberOfInstancesWithMissingValues": calc_number_of_instances_with_missing_values(df, name),
        "NumberOfMissingValues": calc_number_of_missing_values(df, name),
        "NumberOfNumericFeatures": calc_number_of_numeric_features(df, name),
        "NumberOfSymbolicFeatures": calc_number_of_symbolic_features(df, name),

    }

In [91]:
import pickle

with open("/home/cherniak/itmo_job/GAMLET/data/no_meta_features_and_fedot_pipelines_raw/id2dataset_id.pickle", "rb") as f:
    id2dataset_id = pickle.load(f)

In [92]:
import os
from tqdm import tqdm


folds_folder = "/home/cherniak/itmo_job/datasets_folds"

meta_features = {}
for id, dataset_id in tqdm(id2dataset_id.items()):
    dataset_name, dataset_fold = dataset_id.split("_")
    x_fname = f"train_{dataset_name}_fold{dataset_fold}.npy"
    y_fname = f"trainy_{dataset_name}_fold{dataset_fold}.npy"
    x = np.load(os.path.join(folds_folder, x_fname))
    y = np.load(os.path.join(folds_folder, y_fname))
    df = pd.DataFrame(data=x)
    df["y"] = y
    ft = extract(df, dataset_name)
    meta_features[dataset_id] = ft

100%|██████████| 101/101 [00:00<00:00, 144.90it/s]


In [93]:
df = pd.DataFrame.from_dict(meta_features).T
print(df.isna().sum())
df.describe()

MajorityClassSize                     0
MinorityClassSize                     0
NumberOfClasses                       0
NumberOfFeatures                      0
NumberOfInstances                     0
NumberOfInstancesWithMissingValues    0
NumberOfMissingValues                 0
NumberOfNumericFeatures               0
NumberOfSymbolicFeatures              0
dtype: int64


,MajorityClassSize,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
count,101.000000,101.000000,101.000000,101.0,101.000000,101.0,101.0,101.000000,101.000000
mean,936.019802,612.386139,3.217822,15.0,1730.811881,0.0,0.0,95.287129,20.861386
std,678.157209,707.661529,2.256564,0.0,1196.673103,0.0,0.0,254.369154,39.740918
min,108.000000,58.000000,2.000000,15.0,621.000000,0.0,0.0,0.000000,1.000000
25%,345.000000,160.000000,2.000000,15.0,761.000000,0.0,0.0,4.000000,1.000000
50%,630.000000,270.000000,2.000000,15.0,1555.000000,0.0,0.0,9.000000,7.000000
75%,1502.000000,1342.000000,4.000000,15.0,2685.000000,0.0,0.0,20.000000,13.000000
max,2306.000000,2306.000000,10.000000,15.0,4612.000000,0.0,0.0,856.000000,136.000000


In [94]:
to_drop = ["NumberOfInstancesWithMissingValues", "NumberOfMissingValues"]
df = df.drop(to_drop, axis=1)

In [95]:
df.to_csv("/home/cherniak/itmo_job/GAMLET/data/meta_features_and_fedot_pipelines/openml.csv", index=True)